<a href="https://colab.research.google.com/github/rikalestari1411/rikalestari_2006173_tugas_pengolahan_citra/blob/main/Learning_Vector_Quantization(LVQ)_Bunga_Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

iris = pd.read_csv('Iris.csv')
column = ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]
_class = "Species"
iris

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [3]:
del iris['Id']
iris.describe()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [4]:
X, y = iris[column], iris[_class]

# split data into training and test data.
train_X, test_X, train_y, test_y = train_test_split(X, y, 
                                                    train_size=0.5,
                                                    test_size=0.5)

train_y = train_y.to_frame(_class)
test_y = test_y.to_frame(_class)

train = pd.concat([train_X, train_y], axis = 1).values
test = pd.concat([test_X, test_y], axis = 1).values

In [7]:
import numpy as np

class LVQ:
    train_row, train_col, test_col, test_row = 0, 0, 0, 0
    def closest_distance(in_data, weight):
            nrow, ncol = weight.shape
            dist_sum = np.zeros(nrow)
            for i in range(0, nrow):
                dist_sum[i] = np.sum(np.absolute(weight[i, 0:ncol-1]-in_data[0:ncol-1]))
            return np.where(dist_sum == np.min(dist_sum))[0][0] #a one-element array

    def __init__(self, train, test):
        self.train_row, self.train_col = train.shape
        self.test_row, self.test_col = test.shape
        self.classes = np.unique(train[range(0, self.train_row), self.train_col-1])
        
        def encode(data, classes):
            data_row, data_col = data.shape
            
            # Encoding Process
            mapping = {}
            for i in range(0, classes.size):
                mapping[classes[i]] = i
            for i in range(0, data_row):
                data[i, data_col-1] = mapping[data[i, data_col-1]]
            return data
        
        def decode(data, classes):
            data_row, data_col = data.shape
            # Decoding Process
            mapping = {}
            for i in range(0, classes.size):
                mapping[i] = classes[i]
            return mapping
            
        def weight(in_data, n_class):
            nrow, ncol = in_data.shape
            repr_neuron = np.random.random((n_class, ncol))
            
            w_it = 0
            in_data_it = 0
            while (w_it != n_class) & (in_data_it != nrow):
                if (w_it == 0) | (~np.any(in_data[in_data_it, ncol-1] == repr_neuron[:, ncol-1])):
                    repr_neuron[w_it] = in_data[in_data_it]
                    w_it += 1
                in_data_it += 1
            
            return repr_neuron[:,0:ncol] # representative neuron with class
        
        self.decoder = decode(train, self.classes)
        self.train = encode(train, self.classes)
        self.test = encode(test, self.classes)
        self.n_class = np.size(self.classes)
        self.weight = weight(self.train, self.n_class)
        #self.new_weight = np.zeros((self.n_class, self.train_col))
        self.test_result = np.zeros((self.test_row,1))
    def print_meta(self):
        print("Number of row and col (train): " + str(self.train_row) + " " + str(self.train_col) + "\n",  
              "Number of row and col (test): " + str(self.test_row) + " " + str(self.test_col) + "\n", 
              "Number of class: " + str(self.n_class) + "\n",
              "Weight:\n" + str(self.weight) + "\n")
        
    def get_train(self):
        return self.train
    
    def get_test(self):
        return self.test
    
    def learn(self, max_epoch, alpha, threshold):
        def approach(weight, winner, data_in, a):
            wrow, wcol = weight.shape
            incol = data_in.shape[0]
            w, guru = np.hsplit(weight, [wcol-1])
            target = guru[winner]
            cj = data_in[incol-1]

            if target == cj:
                w[winner] = w[winner] + a*(data_in[0:incol-1] - w[winner])
            else:
                w[winner] = w[winner] - a*(data_in[0:incol-1] - w[winner])
    
            return np.hstack([w, guru])
    
        self.alpha = alpha
        self.threshold = threshold
        self.epoch = 0
        while (self.alpha >= self.threshold) | (self.epoch <= max_epoch):
            self.epoch += 1
            beta = self.alpha*(1-(self.epoch/max_epoch))
            for i in range(0, self.train_row):
                self.winner = LVQ.closest_distance(self.train[i], self.weight)
                self.weight = approach(self.weight, self.winner, self.train[i], self.alpha)
            self.alpha *= beta
        return self.weight
    
    def testing(self):
        for i in range(0, self.test_row):
            self.test_result[i,0] = self.weight[LVQ.closest_distance(self.test[i], self.weight), 
                                                self.test_col-1]
        return self.test_result
    
    def get_accuracy(self, print_report = False):
        self.count = 0
        if print_report == True:
            print("Test Result per instance:")
        for i in range(0, self.test_row):
            if print_report == True:
                print(str(i+1) + ". ", "Result: " + str(self.decoder[self.test_result[i, 0]]), 
                      "| Expected: " + str(self.decoder[self.test[i, self.test_col-1]]))
            if self.test_result[i, 0] == self.test[i, self.test_col-1]:
                self.count += 1
        if print_report == True:
            print("--------------------------------------------")
            print(str(self.count) + " of " + str(self.test_row) + " are correctly classified.")
        return (self.count/self.test_row)*100
    def classify(self, input_data, weight = ""):
        # input_data is np array
        if weight == "":
            weight = self.weight
        if len(input_data.shape) == 1:
            in_row = 1
            print(self.decoder[weight[LVQ.closest_distance(input_data, weight), self.test_col-1]])
        else:
            in_row = input_data.shape[0]
            for i in range(0, in_row):
                print(self.decoder[weight[LVQ.closest_distance(input_data[i], weight), self.test_col-1]])

# Example on Iris Dataset
iris_LVQ = LVQ(train, test) # Use this once for building the model

iris_LVQ.print_meta()

iris_LVQ.learn(15, 0.5, 0.000001)

iris_LVQ.print_meta() # The weight is updated

iris_LVQ.testing()

print(iris_LVQ.get_accuracy(print_report = True))

unclassified = np.array([5.6,3.0,4.5,1.5])
iris_LVQ.classify(unclassified) 

Number of row and col (train): 75 5
 Number of row and col (test): 75 5
 Number of class: 3
 Weight:
[[6.4 2.8 5.6 2.1 2. ]
 [5.1 3.5 1.4 0.2 0. ]
 [6.8 2.8 4.8 1.4 1. ]]

Number of row and col (train): 75 5
 Number of row and col (test): 75 5
 Number of class: 3
 Weight:
[[6.90955699 3.11790065 6.16134128 2.31319425 2.        ]
 [4.92659293 3.29864167 1.3974706  0.24507517 0.        ]
 [5.73142251 2.67162561 3.75332068 1.07115522 1.        ]]

Test Result per instance:
1.  Result: Iris-virginica | Expected: Iris-virginica
2.  Result: Iris-virginica | Expected: Iris-virginica
3.  Result: Iris-virginica | Expected: Iris-virginica
4.  Result: Iris-versicolor | Expected: Iris-versicolor
5.  Result: Iris-versicolor | Expected: Iris-versicolor
6.  Result: Iris-setosa | Expected: Iris-setosa
7.  Result: Iris-setosa | Expected: Iris-setosa
8.  Result: Iris-setosa | Expected: Iris-setosa
9.  Result: Iris-versicolor | Expected: Iris-versicolor
10.  Result: Iris-versicolor | Expected: Iris-versi